In [1]:
from pathlib import Path
import re

import pandas as pd
import numpy as np
import spacy
from tqdm.auto import tqdm
import more_itertools
from sklearn.preprocessing import LabelEncoder

tqdm.pandas()
# !python -m spacy download de_core_news_sm

# Exploration and preprocessing

Here I am going to load data.csv into a dataframe, clean it and handle missing values


In [2]:
RESOURCES_PATH = Path("./resources").resolve()
TARGET_COL = "type_subgroup"


df = pd.read_csv(RESOURCES_PATH / "data.csv")
# remove unused index column that was saved during previous pd.to_csv() call
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df = df[df[TARGET_COL] != "Building"].reset_index(drop=True)
df.head(5)

,uid,estate_id,type_id,subtype,price,bedrooms,rooms,bathrooms,total_area,living_area,terrace_area,plot_area,title,features,condition_type,created_at,type_group,type_subgroup
0,2.997918e+08,2.890681e+08,5.0,Einfamilienhaus (freistehend),377750.0,3.0,4.0,1.0,0.0,100.00,0.0,0.0,Viel Wohnraum auf kleiner Fläche.,Gäste-WC; Als Ferienhaus geeignet; Einliegerwo...,new,2022-12-27 16:46:52,house,House
1,1.352158e+08,1.712465e+09,1.0,Mietwohnungen,0.0,0.0,2.0,0.0,0.0,46.56,0.0,0.0,Reserviert • Nicht mehr verfügbar • \n ...,Badewanne; Aufzug; Keller; Etage\n ...,very-good,2021-10-14 16:12:49,apartment,Apartment
2,1.925017e+08,2.880842e+08,8.0,Grundstücke,33350.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,"Land- / Forstwirtschaft in 77839 Lichtenau, Ha...",Provisionsfrei,unknown,2022-03-10 02:19:28,plot,Plot
3,2.639493e+08,2.879910e+08,5.0,Einfamilienhaus (freistehend),362000.0,4.0,5.0,2.0,0.0,181.00,0.0,0.0,Einfamilienhaus mit LWWP und PV-Anlage,Gäste-WC; Provisionsfrei für Kaufende,new,2022-08-30 02:39:21,house,House
4,2.147120e+09,NaN,22.0,Die Ladenfläche,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Laden-/Verkaufsfläche in Stadtfeld-Ost,"Stellplatz, Kategorie Verkaufsfläche Baujahr 1...",used,2024-01-31 15:31:33,investment,Retail


## Let's look at numerical columns


In [3]:
# I have noticed that a lot of rows contain NaN or zero values

numerical_columns = [
    "price",
    "bedrooms",
    "rooms",
    "bathrooms",
    "total_area",
    "living_area",
    "terrace_area",
    "plot_area",
]


print("df length:", df.shape[0])
for column in numerical_columns:
    print(
        f"Column: {column} | NaN values: {df[column].isna().sum()} | Zero values: {(df[column] == 0).sum()}"
    )

df length: 15909
Column: price | NaN values: 30 | Zero values: 7287
Column: bedrooms | NaN values: 123 | Zero values: 11268
Column: rooms | NaN values: 43 | Zero values: 3802
Column: bathrooms | NaN values: 106 | Zero values: 9900
Column: total_area | NaN values: 185 | Zero values: 13637
Column: living_area | NaN values: 36 | Zero values: 3597
Column: terrace_area | NaN values: 201 | Zero values: 15620
Column: plot_area | NaN values: 96 | Zero values: 10211


In [4]:
# I have clarified with you that Zero and NaN values have identical meaning (missing value), Let's replace all with Nan
for column in numerical_columns:
    df[column] = df[column].replace(0, pd.NA)

## Let's look at categorical columns


In [5]:
# I will start from `condition_type` column
# Nothing interesting about this feature. It has only 5 unique values (all normalized lower-case) and doesn't contain NaNs
# "unknown" value probably represent NaNs, let's see later how can we use it)
print("NaN count:", df["condition_type"].isna().sum())
print(df["condition_type"].value_counts())

NaN count: 0
condition_type
used            5358
unknown         5340
very-good       2925
new             1834
to-refurbish     452
Name: count, dtype: int64


In [6]:
# Next column to check is subtype
def clean_scrapped_data(value: str | None) -> str | None:
    """
    A lot of textual data was probably scrapped from the web. It contains a lot of invisible characters
    This function will replace all invisible chars sequences with single space. Return value will be lower cased
    For example 'FOO\n\nbar' -> 'foo bar'
    """
    if pd.isna(value):
        return value
    return re.sub(r"\s+", " ", value).strip().lower()


# let's clean it and keep it for later
df["subtype"] = df["subtype"].progress_apply(clean_scrapped_data)
df["subtype"].value_counts(dropna=False)

  0%|          | 0/15909 [00:00<?, ?it/s]

subtype
NaN                                                                 3346
etagenwohnung                                                       1534
mietwohnungen                                                       1022
häuser zum kauf                                                      975
einfamilienhaus                                                      586
                                                                    ... 
freistehendes einfamilienhaus                                          1
haus zum kauf in neusseinziehen und wohlfühlen - perfekt für die       1
grundstück, industrie                                                  1
einfamilienhaus in schiffdorf 27619                                    1
einfamilienhaus in hofheim 65719                                       1
Name: count, Length: 447, dtype: int64

In [7]:
# Same for `title` column. Let's clean it and keep for later use
# Titles are very different, let's lemmatize it and use most common tokens later


nlp = spacy.load("de_core_news_sm")


def lemmatize_sentences(values: list[str | None]) -> list[list[str]]:
    # replace NaNs with empty strings
    values = [value if not pd.isna(value) else "" for value in values]
    # use spacy pipeline to make it faster for multiple sentences
    docs = nlp.pipe(values)
    tokenized_results = []
    for doc in tqdm(docs, total=len(values)):
        # filter tokens based on their pos
        tokens = [
            token.lemma_.lower()
            for token in doc
            if token.pos_ in {"PROPN", "ADJ", "NOUN"}
        ]
        # Remove non-alphabetic characters
        tokenized_results.append(
            list(
                {
                    clean_token
                    for token in tokens
                    if (clean_token := re.sub("[^A-Za-z0-9_]+", "", token))
                }
            )
        )
    return tokenized_results


df["title"] = df["title"].progress_apply(clean_scrapped_data)
df["title_tokens"] = lemmatize_sentences(df["title"])
title_tokens_count = pd.Series(
    more_itertools.flatten(df["title_tokens"].values)
).value_counts(dropna=False)
print(
    "Most common title tokkens:",
    title_tokens_count,
    sep="\n",
)

  0%|          | 0/15909 [00:00<?, ?it/s]

  0%|          | 0/15909 [00:00<?, ?it/s]

Most common title tokkens:
wohnung               1935
lage                  1361
balkon                1163
zimm                   701
gro                    670
                      ... 
milbertshofenam          1
holzbttg                 1
salzwedel                1
industriehall            1
2zimmercitywohnung       1
Name: count, Length: 12176, dtype: int64


In [8]:
# 'features' column contain textual description. Instead of tokenizing it with spacy,
# let's tokenize it with custom logic - split by delimiters (; \ etc) and clean after


def extract_features(data):
    if pd.isna(data):
        return []

    # fix for cases "etage\n    5"
    data = clean_scrapped_data(data).replace("!", "")

    # split into parts
    delimiters = [";", " - ", "/", ". "]
    parts = re.split("|".join(map(re.escape, delimiters)), data)
    return [f_clean for f in parts if (f_clean := re.sub("[^A-Za-z0-9_]+", "", f))]


df["features_list"] = df["features"].progress_apply(extract_features)
features_count = pd.Series(
    more_itertools.flatten(df["features_list"].values)
).value_counts(dropna=False)
print(
    "Most common features:",
    features_count,
    sep="\n",
)

  0%|          | 0/15909 [00:00<?, ?it/s]

Most common features:
keller                                                             3539
balkon                                                             3141
einbaukche                                                         3080
terrasse                                                           2672
gstewc                                                             2140
                                                                   ... 
nutzflche134m                                                         1
grundstck906m                                                         1
kaufpreis129599jetztfinanzierungsrechnerstarten                       1
einbaukchekategoriedoppelhaushlftebaujahr1998bezugnachabsprache       1
endenergiebedarf11905kwh                                              1
Name: count, Length: 17345, dtype: int64


In [9]:
# Last feature column 'created_at'. Let's extract year, month and day of the week from it
# I have some intuition that day of the week can correlate with type_subgroup

df["created_at"] = pd.to_datetime(df["created_at"])
df["created_year"] = df["created_at"].apply(lambda x: x.year)
df["created_month"] = df["created_at"].apply(lambda x: x.month)
df["created_dayofweek"] = df["created_at"].apply(lambda x: x.dayofweek)

## Let's create feature candidates, see correlation matrix and choose the most suitable features

All possible features are: numerical_features + condition_type + subtype + title_tokens[] + features_list[]

Where X[] is one-hot encoding of sub-features from list X


In [10]:
# Here I decided to implement custom encoder to handle multiple features
# Example: features1: [A, B, C], features2: [C, D] -> features1: [1, 1, 1, 0], features2: [0, 0, 1, 1]
class MultipleFeaturesEncoder:
    def __init__(self, preffix: str, frequency_threshold: int):
        self.preffix = preffix
        # This threshold is used to ignore rare features. They will not be encoded at all
        self.frequency_threshold = frequency_threshold
        self._dictionary = None

    @property
    def column_names(self) -> list[str]:
        if not self._dictionary:
            return []
        return [f"{self.preffix}{token}" for token in self._dictionary]

    def fit(self, values: list[list[str]]):
        value_counts = pd.Series(more_itertools.flatten(values)).value_counts()
        self._dictionary = [
            token
            for token, count in zip(value_counts.index, value_counts.values)
            if count >= self.frequency_threshold
        ]
        return self

    def transform(self, values: list[list[str]]) -> pd.DataFrame:
        if self._dictionary is None:
            raise NotImplementedError
        result = []
        for value in values:
            value = set(value)
            result.append([token in value for token in self._dictionary])
        return pd.DataFrame(result, columns=self.column_names)

    def fit_transform(self, values: list[list[str]]):
        return self.fit(values).transform(values)

In [11]:
# encode features related to extracted from title tokens
# frequency_threshold=100 is mostly random decicion, should be checked later
encoder = MultipleFeaturesEncoder(preffix="TITLE_TOKEN_", frequency_threshold=100)
encoded_title_tokens_df = encoder.fit_transform(df["title_tokens"])
encoded_title_tokens_df

,TITLE_TOKEN_wohnung,TITLE_TOKEN_lage,TITLE_TOKEN_balkon,TITLE_TOKEN_zimm,TITLE_TOKEN_gro,TITLE_TOKEN_ruhig,TITLE_TOKEN_schn,TITLE_TOKEN_garten,TITLE_TOKEN_grundstck,TITLE_TOKEN_einfamilienhaus,...,TITLE_TOKEN_2zimmer,TITLE_TOKEN_bungalow,TITLE_TOKEN_idyllisch,TITLE_TOKEN_efh,TITLE_TOKEN_dachterrasse,TITLE_TOKEN_beliebt,TITLE_TOKEN_3zimmer,TITLE_TOKEN_einbaukche,TITLE_TOKEN_reihenmittelhaus,TITLE_TOKEN_gelegen
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15904,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15905,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
15906,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15907,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
# frequency_threshold=100 is mostly random decicion, should be checked later
encoder = MultipleFeaturesEncoder(preffix="FEATURE_", frequency_threshold=50)
encoded_features_df = encoder.fit_transform(df["features_list"])
encoded_features_df

,FEATURE_keller,FEATURE_balkon,FEATURE_einbaukche,FEATURE_terrasse,FEATURE_gstewc,FEATURE_garage,FEATURE_stellplatz,FEATURE_garten,FEATURE_badewanne,FEATURE_personenaufzug,...,FEATURE_fliesen,FEATURE_energieeffizienzklasseh,FEATURE_monat,FEATURE_energieeffizienzklassef,FEATURE_alsferienhausgeeignet,FEATURE_weitererumekelleranteilwaschtrockenraum,FEATURE_vertragsartmiete,FEATURE_anzahlschlafzimmer3,FEATURE_schlafzimmer4,FEATURE_heizungsartetagenheizung
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,True,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15904,False,True,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
15905,False,False,True,True,True,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
15906,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15907,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [13]:
# Encode categorical features to numbers
condition_type_encoder = LabelEncoder()
df["condition_type_encoded"] = condition_type_encoder.fit_transform(
    df["condition_type"].values
)

subtype_encoder = LabelEncoder()
df["subtype_encoded"] = subtype_encoder.fit_transform(df["subtype"].values)

target_encoder = LabelEncoder()
df["target_encoded"] = target_encoder.fit_transform(df[TARGET_COL].values)

In [14]:
# Here X contains all features that I came up with. I am going to train a baseline model and see feature importance later
X = pd.concat(
    [
        df[numerical_columns + ["condition_type_encoded", "subtype_encoded"]],
        encoded_title_tokens_df,
        encoded_features_df,
    ],
    axis=1,
).replace(pd.NA, 0)
y = df["target_encoded"]
X

/var/folders/mj/_1mmw5h50x98gtlfngvdx3zh0000gn/T/ipykernel_68249/4103643349.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).replace(pd.NA, 0)


,price,bedrooms,rooms,bathrooms,total_area,living_area,terrace_area,plot_area,condition_type_encoded,subtype_encoded,...,FEATURE_fliesen,FEATURE_energieeffizienzklasseh,FEATURE_monat,FEATURE_energieeffizienzklassef,FEATURE_alsferienhausgeeignet,FEATURE_weitererumekelleranteilwaschtrockenraum,FEATURE_vertragsartmiete,FEATURE_anzahlschlafzimmer3,FEATURE_schlafzimmer4,FEATURE_heizungsartetagenheizung
0,377750.0,3.0,4.0,1.0,0.0,100.00,0.0,0.0,0,103,...,False,False,False,False,True,False,False,False,False,False
1,0.0,0.0,2.0,0.0,0.0,46.56,0.0,0.0,4,321,...,False,False,False,False,False,False,False,False,False,False
2,33350.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2,222,...,False,False,False,False,False,False,False,False,False,False
3,362000.0,4.0,5.0,2.0,0.0,181.00,0.0,0.0,0,103,...,False,False,False,False,False,False,False,False,False,False
4,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,3,83,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15904,0.0,0.0,3.0,0.0,0.0,98.00,0.0,0.0,3,446,...,False,False,False,False,False,False,False,False,False,False
15905,289500.0,0.0,7.0,3.0,0.0,172.00,0.0,411.0,3,271,...,False,False,False,False,False,False,False,False,False,False
15906,0.0,13.0,0.0,4.0,0.0,0.00,0.0,0.0,2,355,...,False,False,False,False,False,False,False,False,False,False
15907,0.0,0.0,3.0,1.0,0.0,98.00,0.0,0.0,4,159,...,False,False,False,False,False,False,False,False,False,True


In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

params = {
    "objective": "multi:softmax",
    "num_class": df[TARGET_COL].unique().shape[0],
    "learning_rate": 0.1,
    "max_depth": 10,
    "min_child_weight": 1,
    "gamma": 0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "n_estimators": 100,
    "random_state": 42,
}

# Convert the data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train the XGBoost model
num_round = 10  # Number of boosting rounds
bst = xgb.train(params, dtrain)

# Make predictions
y_pred = bst.predict(dtest)

/Users/ihorbulaievskyi/miniconda3/envs/casafari_test/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:46:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [16]:
from sklearn.metrics import precision_score, recall_score

# Assuming y_test and y_pred contain the true labels and predicted labels, respectively
precision_per_class = precision_score(y_test, y_pred, average=None)
recall_per_class = recall_score(y_test, y_pred, average=None)

# Compute precision and recall for all classes
precision_total = precision_score(y_test, y_pred, average="micro")
recall_total = recall_score(y_test, y_pred, average="micro")

display(
    pd.DataFrame(
        [
            (cls, p, r)
            for cls, p, r in zip(
                target_encoder.classes_, precision_per_class, recall_per_class
            )
        ],
        columns=["class", "precision", "recall"],
    )
)
print("Total Precision:", precision_total)
print("Total Recall:", recall_total)

/Users/ihorbulaievskyi/miniconda3/envs/casafari_test/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,class,precision,recall
0,Apartment,0.834241,0.967509
1,Apartment Building,0.816327,0.677966
2,Bungalow,0.720000,0.500000
3,Chalet,0.000000,0.000000
4,Country Estate,0.000000,0.000000
5,Country House,1.000000,0.153846
6,Duplex,0.933333,0.500000
7,Garage,0.605263,0.766667
8,Hotel,0.000000,0.000000
9,House,0.680827,0.870778


Total Precision: 0.7586423632935261
Total Recall: 0.7586423632935261


# Conclusions

1. It is just an example of how I would start working on this kind of problem. I spent **<span dir="">\~</span>3 hours** in total working on this notebook. It contains initial results and there are **a lot of things that are missing due to limited time**
2. Since the feature set that I came up with is quite sparse, I decided to start from a tree/boosting model. xgboost was a random choice, and the ideal scenario **should include testing different models, tuning hyperparameters, cross-validation training, more feature engineering etc.**
3. I **underestimated** the problematic nature of this data set. Many target classes + high imbalance + a lot of missing values make it difficult to approach this problem with standard methods. How would I approach these problems in case more efforts could be dedicated to this task:
   - **Handling missing values**. Replace missing values with selected statistics such as mean/median/mode. Missing features can be computed by additional small model (KNN, random forest, any boosting model)
   - **Class imbalance.** I could apply resampling techniques - undersampling is not suitable here because the smallest classes have only 1-5 examples in the dataset. Oversampling or SMOTE could be useful here.
4. I worked with a non-english dataset for the first time. I guess I didn't notice a lot of "between-the-lines" information, since I am not familiar with german language. This is an example of situation when you need to invest additional time in order to start feeling the data.

### It was a good excercise to play with, I enjoyed it a lot 😇
